In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

def loadData(code="MSFT", start="2023-01-01", end="2024-01-01"):
    data = yf.download(code, start, end)["Close"]
    return data, code

def signals(data, code):
    data["SMA_20"] = data["Close"].rolling(window=20).mean()
    data["std"] = data.rolling(window=20).std()
    return data, code

def backtestMomentum(data, code, currentCash=1000):
    # if greater than sma buy and if less then sell.
    currentStocksBought = 0
    currentValue = currentCash

    for date, row in data.iterrows():
        if pd.isna(row["SMA_20"]) or pd.isna(row["Close"]):
            continue

        currentPrice = row["Close"]

        sma20 = row["SMA_20"]

        if currentPrice > sma20:
            if currentCash > 0:
                currentStocksBought = currentCash / currentPrice
                currentCash = 0
        else:
            if currentStocksBought > 0:
                currentCash = currentStocksBought * currentPrice
                currentStocksBought = 0

        assetsValue = currentStocksBought * currentPrice 
        currentValue = assetsValue + currentCash
        data.loc[date, "Momentum_Value"] = currentValue # momentum value is like "returns"

    data["Momentum_Value"] = data["Momentum_Value"].astype(float)
    print(f"Final portfolio value: ${currentValue:.2f}")
    metrics(data, "Momentum_Value")

    return data, code

def backtestMeanReversion(data, code, currentCash=1000):
    # if greater than 1.5std*sma then sell. if less than 1.5std*sma then buy (pairs std ratio?)
    initialCash = currentCash
    currentStocksBought = 0
    currentValue = currentCash

    for date, row in data.iterrows():
        if pd.isna(row["SMA_20"]) or pd.isna(row["Close"]):
            continue

        currentPrice = row["Close"]

        sma20 = row["SMA_20"]
        std = row["std"]

        if currentPrice < 1.5 * std * sma20:
            if currentCash > 0:
                currentStocksBought = currentCash / currentPrice
                currentCash = 0
        elif currentPrice > 1.5 * std * sma20:
            if currentStocksBought > 0:
                currentCash = currentStocksBought * currentPrice
                currentStocksBought = 0

        assetsValue = currentStocksBought * currentPrice
        currentValue = assetsValue + currentCash
        data.loc[date, "MeanReversion_Value"] = currentValue

    data["MeanReversion_Value"] = data["MeanReversion_Value"].astype(float)
    print(f"Final portfolio value: ${currentValue:.2f}")
    metrics(data, "MeanReversion_Value")

    return data, code

def metrics(data, column, initialCash=10000):
    finalValue = data[column][-1]
    totalReturns = (finalValue - initialCash) / 100

    data['dailyReturn'] = data[column].pct_change()
    avgReturn = data['dailyReturn'].mean()
    stdReturn = data['dailyReturn'].std()
    riskFreeRate =  0.0001
    sharpeRatio = ((avgReturn - riskFreeRate) / stdReturn) * np.sqrt(252) # 252 trading days a year

    data['rollingMax'] = data[column].cummax()
    data['drawdown'] = data[column] / data['rollingMax'] - 1
    maxDrawdown = data['drawdown'].min()

    print(f"Total Returns: {totalReturns:.2f}")
    print(f"Sharpe Ratio: {sharpeRatio:.2f}")
    print(f"Max Drawdown: {maxDrawdown:.2f}")

    return totalReturns, maxDrawdown, sharpeRatio

def graph(data, code):#TODO
    fig, ax1 = plt.subplots(figsize=(12, 8)) 

    # plot first y-axis
    ax1.plot(data.index, data["Close"], label=code + " Price", color="black")
    ax1.set_xlabel("Date")
    ax1.set_ylabel("Price (USD)")
    ax1.legend(loc="upper left") # legend is the label
    ax1.grid(True)

    # second y axis
    ax2 = ax1.twinx()  
    ax2.plot(data.index, data["Momentum_Value"], label="Momentum Value", color="blue")
    ax2.plot(data.index, data["MeanReversion_Value"], label="Mean Reversion Value", color="red")

    ax2.set_ylabel("Portfolio Value (USD)")
    ax2.legend(loc="upper right")


    plt.title(code + " Momentum vs Mean Reversion Comparison")
    plt.show()

data, code = loadData()
data, code = signals()
data, code = backtestMomentum()
data, code = backtestMeanReversion()
graph(data, code)